In [ ]:
import math
import numpy as np
import pandas as pd

import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils

In [ ]:
!wget https://www.dropbox.com/s/20oe3yly0xtx4vu/archive%20%282%29.zip?dl=0

In [ ]:
# Unzip the dataset, if needed
import zipfile
import pandas as pd
with zipfile.ZipFile('/content/archive (2).zip?dl=0', 'r') as zip_ref:
    zip_ref.extractall('face')

In [ ]:
df = pd.read_csv('/content/face/fer2013/fer2013/fer2013.csv')
print(df.shape)
df.head()

(35887, 3)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
INTERESTED_LABELS = [0,3, 4, 6]

In [ ]:
df = df[df.emotion.isin(INTERESTED_LABELS)]
df.shape

(26217, 3)

In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)

In [ ]:
le = LabelEncoder()
img_labels = le.fit_transform(df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(26217, 4)

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{0: 0, 3: 1, 4: 2, 6: 3}


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.1, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((23595, 48, 48, 1), (2622, 48, 48, 1), (23595, 4), (2622, 4))

In [ ]:
X_train

In [ ]:
del df
del img_array
del img_labels

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
X_train = X_train / 255.
X_valid = X_valid / 255.

In [ ]:
def build_net(optim):
    net = Sequential(name='DCNN')

    net.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            input_shape=(img_width, img_height, img_depth),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_1'))
    net.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_2'
        )
    )
    net.add(BatchNormalization(name='batchnorm_2'))

    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
    net.add(Dropout(0.4, name='dropout_1'))

    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_3'
        )
    )
    net.add(BatchNormalization(name='batchnorm_3'))
    net.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_4'
        )
    )
    net.add(BatchNormalization(name='batchnorm_4'))

    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
    net.add(Dropout(0.4, name='dropout_2'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_5'
        )
    )
    net.add(BatchNormalization(name='batchnorm_5'))
    net.add(
        Conv2D(
            filters=256,
            kernel_size=(3,3),
            activation='elu',
            padding='same',
            kernel_initializer='he_normal',
            name='conv2d_6'
        )
    )
    net.add(BatchNormalization(name='batchnorm_6'))
    net.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_3'))
    net.add(Dropout(0.5, name='dropout_3'))

    net.add(Flatten(name='flatten'))

    net.add(
        Dense(
            128,
            activation='elu',
            kernel_initializer='he_normal',
            name='dense_1'
        )
    )
    net.add(BatchNormalization(name='batchnorm_7'))

    net.add(Dropout(0.6, name='dropout_4'))

    net.add(
        Dense(
            num_classes,
            activation='softmax',
            name='out_layer'
        )
    )
    net.compile(
        loss='categorical_crossentropy',
        optimizer=optim,
        metrics=['accuracy']
    )

    net.summary()

    return net

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
train_datagen.fit(X_train)

In [ ]:
batch_size = 32 #batch size of 32 performs the best.
epochs = 100
optims = [
    optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam'),
    optimizers.Adam(0.001),
]

# I tried both `Nadam` and `Adam`, the difference in results is not different but I finally went with Nadam as it is more popular.
model = build_net(optims[1])
history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / batch_size,
    epochs=epochs,
    callbacks=callbacks,
    use_multiprocessing=True
)

In [ ]:
import pickle
with open( '/content/face.sav', 'wb') as f:
    pickle.dump(model,f)

In [3]:
import pickle
filename = '/content/face (1).sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload() #give the image for detection

In [ ]:
from PIL import Image
import numpy as np
def faceext(path)
    image = Image.open(path)

    image = image.convert("L")

    width = 48  # Define the desired width
    height = 48  # Define the desired height
    image = image.resize((width, height), Image.ANTIALIAS)

    pixel_data = list(image.getdata())

    pixels = np.array(pixel_data)

    reshaped_pixels = pixels.reshape((1, width, height, 1))/255

    return reshaped_pixels



In [ ]:
reshaped_pixels.shape

(1, 48, 48, 1)

In [11]:
def emo(s):
  import numpy as np

# Example array
  predictions = np.array(s, dtype=np.float32)

  # Get the index of the maximum value
  max_index = np.argmax(predictions)

  # Define the labels
  labels = ["anger","happy", "sad", "neutral"]

  # Print the corresponding label based on the max value
  return labels[max_index]



In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
s=loaded_model.predict(reshaped_pixels)
s


1/1 [==============================] - 0s 465ms/step


array([[0.8431294 , 0.08089542, 0.03954441, 0.03643069]], dtype=float32)

In [ ]:
emo(s)

anger


In [9]:
def detect_emotion(face_image):
    # Preprocess the face image
    face_image = cv2.resize(face_image, (48, 48))
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    face_image = np.reshape(face_image, (1, 48, 48, 1))
    face_image = face_image / 255.0

    # Perform emotion prediction
    emotion_prediction = loaded_model.predict(face_image)
    s=emo(emotion_prediction)

    return s


In [ ]:
from google.colab.patches import  cv2_imshow
import cv2

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the video file
video_path = '/content/drive/MyDrive/20230608_190233.mp4'
video = cv2.VideoCapture(video_path)

# Iterate over the video frames
while True:
    # Read a frame from the video
    ret, frame = video.read()

    # Break the loop if the video has ended
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterate over each detected face
    for (x, y, w, h) in faces:
    # Extract the face region
      face_region = frame[y:y+h, x:x+w]

      # Apply face emotion detection
      # Apply face emotion detection
      emotion_label = detect_emotion(face_region)


      # Draw the emotion label on the frame
      text_position = (x, y - 10)
      cv2.putText(frame, emotion_label, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


    # Display the frame with detected faces
    cv2_imshow(frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close any open windows
video.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
from statistics import mode

# Load face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces in a frame
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Function to apply face emotion detection to a face


# Function to calculate the mode value for each time frame
def calculate_mode_values(video_path, time_frames):
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    mode_values = []

    for f,start_frame, end_frame in time_frames:
        start_time = start_frame / frame_rate
        end_time = end_frame / frame_rate

        emotion_values = []
        frame_count = start_frame
        while cap.isOpened() and frame_count <= end_frame:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
            ret, frame = cap.read()

            if not ret:
                break

            faces = detect_faces(frame)
            for (x, y, w, h) in faces:
                face_image = frame[y:y+h, x:x+w]
                emotion = detect_emotion(face_image)
                emotion_values.append(emotion)

            frame_count += 1

        if emotion_values:
            mode_value = mode(emotion_values)
            mode_values.append(mode_value)

    cap.release()
    return mode_values


# Example usage
video_path = '/content/drive/MyDrive/20230608_190233.mp4'
time_frames = [('0', 0, 8.54),
 ('1', 8.54, 10.64),
 ('0', 10.64, 14.12),
 ('1', 14.12, 17.48),
 ('0', 17.48, 21.86),
 ('1', 21.86, 25.16),
 ('0', 25.16, 29.24),
 ('1', 29.24, 32.48),
 ('0', 32.48, 35.48),
 ('1', 35.48, 36.92),
 ('0', 36.92, 39.26),]

result = calculate_mode_values(video_path, time_frames)
for  mode_value in result:
    print(f"Time Frame: {mode_value}")
